In [1]:
import pyaudio
import wave

In [2]:
p = pyaudio.PyAudio()

In [3]:
info = p.get_host_api_info_by_index(0)
numdevices = info.get('deviceCount')

In [4]:
numdevices

19

In [5]:
for i in range(0, numdevices):
    if (p.get_device_info_by_host_api_device_index(0, i).get('maxInputChannels')) > 0:
        print("Input Device id ", i, " - ", p.get_device_info_by_host_api_device_index(0, i).get('name'))

Input Device id  1  -  tegra-snd-t210ref-mobile-rt565x: - (hw:1,0)
Input Device id  2  -  tegra-snd-t210ref-mobile-rt565x: - (hw:1,1)
Input Device id  3  -  tegra-snd-t210ref-mobile-rt565x: - (hw:1,2)
Input Device id  4  -  tegra-snd-t210ref-mobile-rt565x: - (hw:1,3)
Input Device id  5  -  tegra-snd-t210ref-mobile-rt565x: - (hw:1,4)
Input Device id  6  -  tegra-snd-t210ref-mobile-rt565x: - (hw:1,5)
Input Device id  7  -  tegra-snd-t210ref-mobile-rt565x: - (hw:1,6)
Input Device id  8  -  tegra-snd-t210ref-mobile-rt565x: - (hw:1,7)
Input Device id  9  -  tegra-snd-t210ref-mobile-rt565x: - (hw:1,8)
Input Device id  10  -  tegra-snd-t210ref-mobile-rt565x: - (hw:1,9)
Input Device id  11  -  USB Composite Device: Audio (hw:2,0)
Input Device id  13  -  pulse
Input Device id  14  -  music
Input Device id  16  -  voice
Input Device id  17  -  music_and_voice
Input Device id  18  -  default


In [6]:
devinfo = p.get_device_info_by_index(11)  # Or whatever device you care about.
if p.is_format_supported(48000.0,  # Sample rate
                         input_device=devinfo['index'],
                         input_channels=devinfo['maxInputChannels'],
                         input_format=pyaudio.paInt16):
    print('Yay!')

Yay!


In [7]:
import sounddevice as sd

samplerates = 32000, 44100, 48000, 96000, 128000
device = 11

supported_samplerates = []
for fs in samplerates:
    try:
        sd.check_output_settings(device=device, samplerate=fs)
    except Exception as e:
        print(fs, e)
    else:
        supported_samplerates.append(fs)
print(supported_samplerates)

32000 Invalid number of channels [PaErrorCode -9998]
44100 Invalid number of channels [PaErrorCode -9998]
48000 Invalid number of channels [PaErrorCode -9998]
96000 Invalid number of channels [PaErrorCode -9998]
128000 Invalid number of channels [PaErrorCode -9998]
[]


In [12]:
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 48000
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "output.wav"

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                input_device_index=11,
                frames_per_buffer=CHUNK)

print("* recording")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK, exception_on_overflow = False)
    frames.append(data)

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

* recording
* done recording
